## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly.**

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [3]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the cell below to find the number of rows in the dataset.

In [4]:
df.shape

(294478, 5)

c. The number of unique users in the dataset.

In [5]:
# df.user_id.unique()
df.user_id.nunique()

290584

d. The proportion of users converted.

In [6]:
# df.query('converted == 1').user_id.mean()
df.converted.mean()

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't match.

In [7]:
df.query("group == 'treatment' and not landing_page == 'new_page' or group == 'control' and landing_page == 'new_page'").count()[0]

3893

f. Do any of the rows have missing values?

In [8]:
df.isnull().sum() 

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [9]:
df2 = df.copy()

In [10]:
no_matches = df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].index

In [11]:
df2.drop(no_matches, inplace=True)

In [12]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

In [13]:
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [14]:
df2.user_id.nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [15]:
df2[df2.user_id.duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


c. What is the row information for the repeat **user_id**? 

In [16]:
df2[df2.user_id.duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [17]:
duplicate_user = df2[df2.user_id.duplicated()].index # save duplicated row index to variable name 

In [18]:
df2.drop(duplicate_user, inplace=True) # drop duplicated user_id row 

In [19]:
df2[df2.user_id.duplicated()] # check to confirm duplicate removed

,user_id,timestamp,group,landing_page,converted


`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [20]:
df2.query("converted == 1").count()[0]/ df2.converted.shape[0]*100

11.959708724499627

b. Given that an individual was in the `control` group, what is the probability they converted?

In [21]:
df2.query("group == 'control' and converted == 1").count()[0] /df2.query("group == 'control'").count()[0]*100

12.038630450046121

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [22]:
df2.query("group == 'treatment' and converted == 1").count()[0]/ df2.query("group == 'treatment'").count()[0]*100

11.880806551510565

d. What is the probability that an individual received the new page?

In [23]:
df2.query("landing_page == 'new_page'").count()[0] / df2.landing_page.shape[0]*100

50.00619442226688

In [24]:
df2[df2['converted']==1].count()[0] / df2.converted.shape[0]*100

11.959708724499627

In [25]:
df2[df2['converted']==0].count()[0] / df2.converted.shape[0]*100

88.040291275500365

In [26]:
treatment = df2[df2['group'] == 'treatment']['converted'].mean()
control =  df2[df2['group'] == 'control']['converted'].mean()
obs_diff = control - treatment # observed differnces in the means
obs_diff*100

0.15782389853555567

In [27]:
import datetime
df2.timestamp.max(), df.timestamp.min() # getting min, max of timestamps

('2017-01-24 13:41:54.460509', '2017-01-02 13:42:05.378582')

In [28]:
df2.timestamp = pd.to_datetime(df2['timestamp']) # group by sort timestamp 
df2.timestamp.astype(str) 
df2.groupby('timestamp')

In [34]:
df2.groupby('timestamp')
df2.sort_values('timestamp', ascending=False)

,user_id,timestamp,group,landing_page,converted
193652,920411,2017-01-24 13:41:54.460509,control,old_page,0
179072,836373,2017-01-24 13:41:52.604673,control,old_page,0
157787,843121,2017-01-24 13:41:44.097174,treatment,new_page,0
47535,808330,2017-01-24 13:41:19.152664,control,old_page,0
158814,799244,2017-01-24 13:41:15.325359,treatment,new_page,0
180924,808180,2017-01-24 13:41:07.867394,control,old_page,0
29690,787704,2017-01-24 13:41:04.416938,treatment,new_page,0
260161,712000,2017-01-24 13:40:54.479759,control,old_page,1
178372,891992,2017-01-24 13:40:37.103733,control,old_page,0
24692,725007,2017-01-24 13:40:23.844184,control,old_page,0


In [29]:
end = '2017-01-24'
start = '2017-01-02'       

In [30]:
from datetime import datetime
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [31]:
days_between(start, end) # get duration of A/B test period

22

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

In this dataset which spanned over the course of 22 days, we had `290584` unique users, where the probability that they convert regardless of what page they recieved either, `landing_page` or `old_page`, was `11.95%`. If the user was in the `control` group the probability they'd convert was `12.03%` while if the user was in the `treatment` group their probability of conversion was `11.88%`. Finally the probability the user received the `new_page` was `50%`. 

Calculating the `observed difference` between the conversion rates we get about a `0.0015` difference which is minimal but, in order to deem `statistically significant` we would have to perform a `bootstrap` on the sample to _"simulate the sampling distributions for the difference in proportions" (cite 1: Udacity, Lesson 13: Video #7 "Metric Click Through Rate")_

With these findings it is difficult to say which page leads were more effective. 

As we learned in the `A/B Tests` lesson there are important questions to consider when concluding results from our statistics. We haven't yet introduced `Hypothesis Testing` also in which we could introduce sampling, and other metrics.  

In conclusion, the next best option is to continue with the testing but, for a longer duration to rule out `response bias` or `change aversion` and begin with `Hypothesis Testing` to look at new metrics like p values, confidence intervals, etc. 

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

1. **Null Hypothesis**: 
> $$H_0:   p_{old}  \leq   p_{new} >     Type I Error Rate .05 $$
2. **Alternative Hypothesis**: 
> $$H_1: p_{new}  \geq  p_{old}  < Type I Error Rate of .05 $$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

1. **Null Hypothesis**: 
> $$H_0:  \mu_{old} - \mu_{new} = 0$$
2. **Alternative Hypothesis**: 
> $$H_1: \mu_{old} - \mu_{new} \neq 0$$

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [33]:
# df2[df2['converted']==1].count()[0] / df2.converted.shape[0]*100
df.query("landing_page == 'new_page' and converted == 1").count()[0] /df.query("landing_page == 'new_page'").count()[0]*100

11.884079625642663

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [34]:
df.query("landing_page == 'old_page' and converted == 1").count()[0] /df.query("landing_page == 'old_page'").count()[0]*100

12.047759085568362

c. What is $n_{new}$, the number of individuals in the treatment group?

In [39]:
df.query("group == 'treatment'").user_id.nunique()
# df.query('action == "click"').id.nunique() / df.query('action == "view"').id.nunique()

146284

d. What is $n_{old}$, the number of individuals in the control group?

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.randint(2, size=(int(1e6), 3))

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []
for _ in range(10000):
    boot_sample = coffee_red.sample(200, replace = True)
    boot_mean = boot_sample[boot_sample['converted'] == 1]['user_id'].mean()
    p_diffs.append(boot_mean)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs); 

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = 
convert_new = 
n_old = 
n_new = 

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

**Put your answer here.**

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Put your answer here.**

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

<a id='conclusions'></a>
## Finishing Up

> Congratulations!  You have reached the end of the A/B Test Results project!  You should be very proud of all you have accomplished!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])